<a href="https://colab.research.google.com/github/aboagyeso/MLP-FFNN/blob/master/Copy_of_Kinase_Regression_P49841.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORT PACKAGES

In [1]:
import numpy as np
import sklearn.metrics, math
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor as rf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import KFold
from sklearn import metrics
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from tensorflow import set_random_seed
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy as np
# Set random seed
np.random.seed(0)

Using TensorFlow backend.


In [0]:

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error 
import seaborn as sns

import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
#from xgboost import XGBRegressor

DEFINE CUSTOM FUNCTIONS

In [0]:


# root mean squared error (rmse) for regression
def rmse(y_true, y_pred):
    from keras import backend
    return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

# mean squared error (mse) for regression
def mse(y_true, y_pred):
    from keras import backend
    return backend.mean(backend.square(y_pred - y_true), axis=-1)

# coefficient of determination (R^2) for regression
def r_square(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return (1 - SS_res/(SS_tot + K.epsilon()))

def r_square_loss(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return 1 - ( 1 - SS_res/(SS_tot + K.epsilon()))

In [0]:

def add_gausian(y,mu,sigma):
    noise = np.random.normal(mu,sigma,[1412, 117])
    df_aug = df_train_set +noise 
    #df_new = pd.concat([df_train_set,df_aug])
    return(df_aug)

def drop_corr(df):
    corr_matrix = df.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(np.bool))
    to_drop = [column for column in upper.columns if any(upper[column]>0.95)]
    df_new = df.drop(df[to_drop],axis = 1)
    return(df_new)

def model_eval(model):
    model.fit(X_train,y_train)
    pred = model.predict(X_test)
    #model_score = model.score(X_test,y_test)
    model_mse =  mse(y_test,pred)
    model_r2 =r2(y_test,pred)
    return(-model_mse, model_r2)

In [0]:
# Load libraries
from keras import models
from keras import layers
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

READ IN THE DATAFRAME AND CHECK OUTPUT

In [6]:
df = pd.read_csv('https://raw.githubusercontent.com/aboagyeso/Kinase_activity/master/data/kinase_files/KinaseP49841.csv')
df.head()

,Unnamed: 0,target_id,pAc,smiles,SlogP,SMR,LabuteASA,TPSA,AMW,ExactMW,NumLipinskiHBA,NumLipinskiHBD,NumRotatableBonds,NumHBD,NumHBA,NumAmideBonds,NumHeteroAtoms,NumHeavyAtoms,NumAtoms,NumStereocenters,NumUnspecifiedStereocenters,NumRings,NumAromaticRings,NumSaturatedRings,NumAliphaticRings,NumAromaticHeterocycles,NumSaturatedHeterocycles,NumAliphaticHeterocycles,NumAromaticCarbocycles,NumSaturatedCarbocycles,NumAliphaticCarbocycles,FractionCSP3,Chi0v,Chi1v,Chi2v,Chi3v,Chi4v,Chi1n,Chi2n,Chi3n,...,MQN2,MQN3,MQN4,MQN5,MQN6,MQN7,MQN8,MQN9,MQN10,MQN11,MQN12,MQN13,MQN14,MQN15,MQN16,MQN17,MQN19,MQN20,MQN21,MQN22,MQN23,MQN24,MQN25,MQN26,MQN27,MQN28,MQN29,MQN30,MQN31,MQN32,MQN33,MQN34,MQN35,MQN36,MQN37,MQN38,MQN39,MQN40,MQN41,MQN42
0,27604,P49841,1.30,C1COCCN1CCNC2=NC3=CC=CC=C3C4=C2C5=CC=CC=C5N4,3.61340,106.8544,151.512268,53.18,346.434,346.179361,5,2,4,2,4,0,5,26,48,0,0,5,4,1,1,2,1,1,2,0,0,0.285714,14.664118,9.231501,6.778933,5.172699,3.833922,9.231501,6.778933,5.172699,...,0,0,0,0,0,0,1,3,0,1,26,4,0,0,16,10,4,6,5,2,2,0,0,0,3,0,0,15,8,0,0,0,1,4,0,0,0,0,6,3
1,27605,P49841,1.30,CCN(CC1=CN2C=CC=CC2=N1)CC3=NC(=O)C4=CC(=C(C=C4...,2.61010,110.4477,167.285365,84.75,393.447,393.180090,8,1,7,1,7,0,8,29,52,0,0,4,4,0,0,3,0,0,1,0,0,0.285714,16.676371,9.408248,6.743048,4.889656,3.384215,9.408248,6.743048,4.889656,...,0,0,0,0,0,0,1,4,3,0,29,10,1,0,11,10,7,11,8,1,1,0,0,4,5,1,0,10,9,0,0,0,1,3,0,0,0,0,4,2
2,27606,P49841,2.44,COC1=CC=C(C=C1)CNC(=O)NC2=NC3=C(S2)C=C(C=C3)[N...,3.53480,95.1268,146.152767,106.39,358.379,358.073576,8,2,5,2,6,2,9,25,39,0,0,3,3,0,0,1,0,0,2,0,0,0.125000,14.000724,8.093718,6.114557,4.349027,3.041326,7.277221,5.083908,3.415426,...,0,0,0,0,1,0,3,1,4,0,25,9,2,0,8,8,5,13,8,2,2,1,1,4,4,2,0,9,6,0,0,0,1,2,0,0,0,0,2,1
3,27607,P49841,2.76,CCCC(=O)NC1=NNC2=CC(=C(C=C21)C3=CC=CC=C3)C4=CC...,5.63550,110.5034,157.795671,57.78,355.441,355.168462,4,2,5,2,2,1,4,27,48,0,0,4,4,0,0,1,0,0,3,0,0,0.130435,15.197879,9.188066,6.545816,4.605439,3.294453,9.188066,6.545816,4.605439,...,0,0,0,0,0,0,1,2,1,0,27,7,1,0,11,11,5,5,4,2,2,0,0,2,3,1,0,14,7,0,0,0,1,3,0,0,0,0,2,1
4,27608,P49841,2.84,COC1=CC=C(C=C1)CNC(=O)NC2=NC=CC(=C2)C#N,2.28358,77.8044,121.935285,87.04,282.303,282.111676,6,2,4,2,4,2,6,21,35,0,0,2,2,0,0,1,0,0,1,0,0,0.133333,11.459482,6.249742,4.197313,2.742760,1.654688,6.249742,4.197313,2.742760,...,0,0,0,0,0,0,3,1,2,0,21,8,1,1,6,6,4,8,6,2,2,0,0,3,5,1,0,8,4,0,0,0,0,2,0,0,0,0,0,0


In [7]:
#Check the shape of the dataframe
df.shape

(1569, 120)

In [8]:
#Drop the highly correlated features
drop_corr(df).head()

,Unnamed: 0,target_id,smiles,SlogP,SMR,TPSA,NumLipinskiHBA,NumLipinskiHBD,NumRotatableBonds,NumHBA,NumAmideBonds,NumHeteroAtoms,NumStereocenters,NumUnspecifiedStereocenters,NumRings,NumAromaticRings,NumSaturatedRings,NumAliphaticRings,NumAromaticHeterocycles,NumSaturatedHeterocycles,NumAliphaticHeterocycles,NumAromaticCarbocycles,NumSaturatedCarbocycles,FractionCSP3,HallKierAlpha,slogp_VSA1,slogp_VSA2,slogp_VSA3,slogp_VSA4,slogp_VSA5,slogp_VSA6,slogp_VSA7,slogp_VSA8,slogp_VSA10,slogp_VSA11,slogp_VSA12,smr_VSA1,smr_VSA2,smr_VSA3,smr_VSA4,...,peoe_VSA8,peoe_VSA9,peoe_VSA10,peoe_VSA11,peoe_VSA12,peoe_VSA13,peoe_VSA14,MQN2,MQN3,MQN4,MQN5,MQN6,MQN7,MQN8,MQN9,MQN10,MQN11,MQN13,MQN14,MQN15,MQN16,MQN20,MQN24,MQN26,MQN27,MQN28,MQN29,MQN30,MQN31,MQN32,MQN33,MQN34,MQN35,MQN36,MQN37,MQN38,MQN39,MQN40,MQN41,MQN42
0,27604,P49841,C1COCCN1CCNC2=NC3=CC=CC=C3C4=C2C5=CC=CC=C5N4,3.61340,106.8544,53.18,5,2,4,4,0,5,0,0,5,4,1,1,2,1,1,2,0,0.285714,-2.63,5.316789,54.260656,4.736863,0.000000,0.000000,48.530937,0.0,32.708775,5.817863,0.000000,0.000000,4.736863,0.000000,14.867867,0.0,...,42.468175,29.633390,5.817863,0.000000,0.00000,0.000000,0.000000,0,0,0,0,0,0,1,3,0,1,4,0,0,16,6,0,0,3,0,0,15,8,0,0,0,1,4,0,0,0,0,6,3
1,27605,P49841,CCN(CC1=CN2C=CC=CC2=N1)CC3=NC(=O)C4=CC(=C(C=C4...,2.61010,110.4477,84.75,8,1,7,7,0,8,0,0,4,4,0,0,3,0,0,1,0,0.285714,-3.39,15.032993,45.016891,13.089513,0.000000,18.442070,47.520060,0.0,16.550102,0.000000,11.499024,0.000000,9.473726,0.000000,24.252540,0.0,...,25.004811,37.361204,11.471582,11.499024,0.00000,5.559267,0.000000,0,0,0,0,0,0,1,4,3,0,10,1,0,11,11,0,4,5,1,0,10,9,0,0,0,1,3,0,0,0,0,4,2
2,27606,P49841,COC1=CC=C(C=C1)CNC(=O)NC2=NC3=C(S2)C=C(C=C3)[N...,3.53480,95.1268,106.39,8,2,5,6,2,9,0,0,3,3,0,0,1,0,0,2,0,0.125000,-3.20,15.370440,23.048202,6.544756,10.114318,5.563451,42.464569,0.0,10.216621,15.613482,5.749512,11.336786,14.454711,0.000000,10.300767,0.0,...,18.677491,22.249729,5.749512,5.131558,0.00000,5.687386,6.031115,0,0,0,0,1,0,3,1,4,0,9,2,0,8,13,1,4,4,2,0,9,6,0,0,0,1,2,0,0,0,0,2,1
3,27607,P49841,CCCC(=O)NC1=NNC2=CC(=C(C=C21)C3=CC=CC=C3)C4=CC...,5.63550,110.5034,57.78,4,2,5,2,1,4,0,0,4,4,0,0,1,0,0,3,0,0.130435,-3.40,5.316789,16.104543,4.794537,0.000000,19.765380,72.796405,0.0,33.156731,5.817863,0.000000,0.000000,4.794537,0.000000,10.197364,0.0,...,11.807046,5.516701,0.000000,5.817863,5.90718,0.000000,0.000000,0,0,0,0,0,0,1,2,1,0,7,1,0,11,5,0,2,3,1,0,14,7,0,0,0,1,3,0,0,0,0,2,1
4,27608,P49841,COC1=CC=C(C=C1)CNC(=O)NC2=NC=CC(=C2)C#N,2.28358,77.8044,87.04,6,2,4,4,2,6,0,0,2,2,0,0,1,0,0,1,0,0.133333,-3.07,15.370440,18.124891,6.544756,11.331113,11.126903,42.595046,0.0,0.000000,10.612400,5.749512,0.000000,9.531400,5.261892,10.300767,0.0,...,12.741600,18.742470,11.567375,0.000000,0.00000,0.000000,6.031115,0,0,0,0,0,0,3,1,2,0,8,1,1,6,8,0,3,5,1,0,8,4,0,0,0,0,2,0,0,0,0,0,0


In [9]:
drop_corr(df).shape

(1569, 92)

In [0]:
#Plot distribution of the pAc with the density estimation function
#f, ax = plt.subplots(figsize=(6, 6))
#sns.distplot(df.pAc,color = 'green');
#sns.set_style("whitegrid")

In [0]:
#Boxplot of pAc,It has a few outliers but the distribution in fairly normal
#f, ax = plt.subplots(figsize=(5, 6))
#sns.set_style("whitegrid")
#data = df.pAc
#sns.boxplot(data=data);

In [12]:
#Drop unwanted columns
df = df.drop(['Unnamed: 0','smiles','target_id'], axis=1)
df.head()

,pAc,SlogP,SMR,LabuteASA,TPSA,AMW,ExactMW,NumLipinskiHBA,NumLipinskiHBD,NumRotatableBonds,NumHBD,NumHBA,NumAmideBonds,NumHeteroAtoms,NumHeavyAtoms,NumAtoms,NumStereocenters,NumUnspecifiedStereocenters,NumRings,NumAromaticRings,NumSaturatedRings,NumAliphaticRings,NumAromaticHeterocycles,NumSaturatedHeterocycles,NumAliphaticHeterocycles,NumAromaticCarbocycles,NumSaturatedCarbocycles,NumAliphaticCarbocycles,FractionCSP3,Chi0v,Chi1v,Chi2v,Chi3v,Chi4v,Chi1n,Chi2n,Chi3n,Chi4n,HallKierAlpha,kappa1,...,MQN2,MQN3,MQN4,MQN5,MQN6,MQN7,MQN8,MQN9,MQN10,MQN11,MQN12,MQN13,MQN14,MQN15,MQN16,MQN17,MQN19,MQN20,MQN21,MQN22,MQN23,MQN24,MQN25,MQN26,MQN27,MQN28,MQN29,MQN30,MQN31,MQN32,MQN33,MQN34,MQN35,MQN36,MQN37,MQN38,MQN39,MQN40,MQN41,MQN42
0,1.30,3.61340,106.8544,151.512268,53.18,346.434,346.179361,5,2,4,2,4,0,5,26,48,0,0,5,4,1,1,2,1,1,2,0,0,0.285714,14.664118,9.231501,6.778933,5.172699,3.833922,9.231501,6.778933,5.172699,3.833922,-2.63,15.611372,...,0,0,0,0,0,0,1,3,0,1,26,4,0,0,16,10,4,6,5,2,2,0,0,0,3,0,0,15,8,0,0,0,1,4,0,0,0,0,6,3
1,1.30,2.61010,110.4477,167.285365,84.75,393.447,393.180090,8,1,7,1,7,0,8,29,52,0,0,4,4,0,0,3,0,0,1,0,0,0.285714,16.676371,9.408248,6.743048,4.889656,3.384215,9.408248,6.743048,4.889656,3.384215,-3.39,18.949471,...,0,0,0,0,0,0,1,4,3,0,29,10,1,0,11,10,7,11,8,1,1,0,0,4,5,1,0,10,9,0,0,0,1,3,0,0,0,0,4,2
2,2.44,3.53480,95.1268,146.152767,106.39,358.379,358.073576,8,2,5,2,6,2,9,25,39,0,0,3,3,0,0,1,0,0,2,0,0,0.125000,14.000724,8.093718,6.114557,4.349027,3.041326,7.277221,5.083908,3.415426,2.147375,-3.20,16.650576,...,0,0,0,0,1,0,3,1,4,0,25,9,2,0,8,8,5,13,8,2,2,1,1,4,4,2,0,9,6,0,0,0,1,2,0,0,0,0,2,1
3,2.76,5.63550,110.5034,157.795671,57.78,355.441,355.168462,4,2,5,2,2,1,4,27,48,0,0,4,4,0,0,1,0,0,3,0,0,0.130435,15.197879,9.188066,6.545816,4.605439,3.294453,9.188066,6.545816,4.605439,3.294453,-3.40,17.035921,...,0,0,0,0,0,0,1,2,1,0,27,7,1,0,11,11,5,5,4,2,2,0,0,2,3,1,0,14,7,0,0,0,1,3,0,0,0,0,2,1
4,2.84,2.28358,77.8044,121.935285,87.04,282.303,282.111676,6,2,4,2,4,2,6,21,35,0,0,2,2,0,0,1,0,0,1,0,0,0.133333,11.459482,6.249742,4.197313,2.742760,1.654688,6.249742,4.197313,2.742760,1.654688,-3.07,14.341447,...,0,0,0,0,0,0,3,1,2,0,21,8,1,1,6,6,4,8,6,2,2,0,0,3,5,1,0,8,4,0,0,0,0,2,0,0,0,0,0,0


In [0]:
# Get the feature vector
X = df.drop(columns=['pAc'],axis=1)
# Get the target vector
y = df["pAc"]

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,random_state=42)

In [15]:
#Check the shape of the training and testing data 
X_train.shape, X_test.shape, y_train.shape, y_test.shape 

((1412, 116), (157, 116), (1412,), (157,))

 Add the X_train and y_train to form a a training set for augmentation
 **bold text**

In [16]:
df_train_set = pd.concat([X_train, y_train], axis = 1)
df_train_set.head()

,SlogP,SMR,LabuteASA,TPSA,AMW,ExactMW,NumLipinskiHBA,NumLipinskiHBD,NumRotatableBonds,NumHBD,NumHBA,NumAmideBonds,NumHeteroAtoms,NumHeavyAtoms,NumAtoms,NumStereocenters,NumUnspecifiedStereocenters,NumRings,NumAromaticRings,NumSaturatedRings,NumAliphaticRings,NumAromaticHeterocycles,NumSaturatedHeterocycles,NumAliphaticHeterocycles,NumAromaticCarbocycles,NumSaturatedCarbocycles,NumAliphaticCarbocycles,FractionCSP3,Chi0v,Chi1v,Chi2v,Chi3v,Chi4v,Chi1n,Chi2n,Chi3n,Chi4n,HallKierAlpha,kappa1,kappa2,...,MQN3,MQN4,MQN5,MQN6,MQN7,MQN8,MQN9,MQN10,MQN11,MQN12,MQN13,MQN14,MQN15,MQN16,MQN17,MQN19,MQN20,MQN21,MQN22,MQN23,MQN24,MQN25,MQN26,MQN27,MQN28,MQN29,MQN30,MQN31,MQN32,MQN33,MQN34,MQN35,MQN36,MQN37,MQN38,MQN39,MQN40,MQN41,MQN42,pAc
162,4.45602,142.3172,223.806917,65.77,532.570,532.219844,7,1,4,1,6,1,10,39,66,0,0,5,4,1,1,2,1,1,2,0,0,0.275862,21.587246,12.510280,9.620033,6.765781,4.449942,12.510280,9.620033,6.765781,4.449942,-4.20,26.408832,10.884506,...,0,0,0,0,0,1,5,1,0,39,13,1,1,17,11,4,8,7,1,1,0,0,6,4,1,1,16,11,0,0,0,1,4,0,0,0,0,2,1,4.91
1134,2.70330,107.4078,171.701514,126.05,406.446,406.175339,9,3,4,3,8,0,9,30,52,2,2,7,3,4,4,2,0,0,1,4,4,0.476190,16.322836,10.332030,9.320915,7.029169,6.347269,10.332030,9.320915,7.029169,6.347269,-3.07,17.760024,5.975998,...,0,0,0,0,0,1,5,3,0,30,6,1,0,20,8,4,12,9,3,3,0,0,3,1,1,0,15,8,2,0,0,1,6,0,0,0,0,12,13,7.29
352,6.18650,170.7687,248.562788,72.16,565.677,565.247775,7,1,7,1,6,2,7,43,74,0,0,8,6,1,2,2,1,2,4,0,0,0.194444,23.792703,15.034545,11.485827,8.861406,6.789995,15.034545,11.485827,8.861406,6.789995,-5.01,25.680648,10.364729,...,0,0,0,0,0,0,5,2,0,43,7,2,0,25,16,7,9,7,1,1,0,0,2,3,0,0,22,16,0,0,0,4,4,0,0,0,0,6,3,5.60
518,3.85070,57.8435,93.565816,17.07,362.868,359.745472,1,0,2,0,2,0,5,11,14,0,0,1,1,0,0,1,0,0,0,0,0,0.166667,10.809333,6.212464,5.574269,4.847912,3.010652,2.688502,1.789100,1.041344,0.549852,0.81,9.894674,4.161980,...,0,3,0,1,0,0,0,1,0,11,5,1,0,3,2,2,2,1,0,0,0,0,4,1,1,0,2,3,0,0,0,1,0,0,0,0,0,0,0,6.00
44,1.19987,59.4347,90.705947,50.78,221.285,221.062283,4,1,2,1,5,0,5,15,26,0,0,2,2,0,0,1,0,0,1,0,0,0.200000,9.121278,5.442664,4.054747,3.096280,2.149967,4.669268,3.044701,2.105998,1.387221,-1.62,9.916984,3.860084,...,0,0,0,1,0,1,2,1,0,15,3,2,0,6,5,2,5,4,1,1,0,0,3,1,0,0,6,5,0,0,0,1,1,0,0,0,0,0,0,4.19


In [0]:
#Assign df0 to df_train_set
df0 = df_train_set

In [18]:
#The shape of the training set
df0.shape

(1412, 117)

DATA BEFORE AUGMENTATION

In [19]:
# Get the feature vector
X0 = df0.drop(columns=['pAc'],axis=1).values
# Get the target vector
y0 = df0["pAc"].values
X0_train, X0_test, y0_train, y0_test = train_test_split(X0, y0, test_size=0.3,random_state=42)
X0_train.shape,y0_train.shape,X0_test.shape,y0_test.shape

((988, 116), (988,), (424, 116), (424,))

VALIDATION SET FOR EXTERNAL VALIDATION

In [0]:
X_test_val = X_test.values
y_test_val = y_test.values

AUGMENTATION 0.1

In [0]:
#Add noise of 0.01 to the dataframe
noise = add_gausian(df0,0,0.1)
df1 = pd.concat([df0,noise])
df1.shape
# Get the feature vector
X1 = df1.drop(columns=['pAc'],axis=1).values
# Get the target vector
y1 = df1["pAc"].values

#Split data into training and testing set
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.3,random_state=42)

AUGMENTATION 0.2

In [0]:
#Add noise of 0.02 to the dataframe
noise = add_gausian(df0,0,0.2)
df2=pd.concat([df1,noise])

# Get the feature vector
X2 = df2.drop(columns=['pAc'],axis=1).values
# Get the target vector
y2 = df2["pAc"].values
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.3,random_state=42)

AUGMENTATION 0.3

In [0]:
#Add noise of 0.03 to the dataframe
noise3 = add_gausian(df0,0,0.3)

df3 = pd.concat([df2,noise3])
# Get the feature vector
#X = df.drop('smiles', 'mol', 'pAc', axis=1)
X3 = df3.drop(columns=['pAc'],axis=1).values
# Get the target vector
y3 = df3["pAc"].values
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=0.3,random_state=42)

BUILD DEEP LEARNING MODELS


MODEL0

In [24]:
model0 = Sequential()
# The Input Layer :
model0.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))

# The Hidden Layers :
model0.add(Dense(256, kernel_initializer='normal',activation='relu'))
model0.add(Dense(256, kernel_initializer='normal',activation='relu'))
model0.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
model0.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
model0.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])
#Train the model
model0.fit(X0_train, y0_train, epochs=200, batch_size=32, validation_split = 0.2,verbose=0)



W0728 20:28:16.570930 140612495423360 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0728 20:28:16.655900 140612495423360 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0728 20:28:16.664192 140612495423360 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4115: The name tf.random_normal is deprecated. Please use tf.random.normal instead.

W0728 20:28:17.182969 140612495423360 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0728 20:28:18.235135 140612495423360 deprecation_wrapper.

MODEL 1

In [25]:
model1= Sequential()
# The Input Layer :
model1.add(Dense(128, kernel_initializer='normal',input_dim = X1_train.shape[1], activation='relu'))

# The Hidden Layers :
model1.add(Dense(256, kernel_initializer='normal',activation='relu'))
model1.add(Dense(256, kernel_initializer='normal',activation='relu'))
model1.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
model1.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
model1.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])
#Train the model
model1.fit(X1_train, y1_train, epochs=200, batch_size=32, validation_split = 0.2,verbose=0)

MODEL 2

In [26]:
np.random.seed(7)
# set the TF seed
set_random_seed(7)
# built Keras sequential model 
model2 = Sequential()
# add batch normalization
model2.add(BatchNormalization())
# add layer to the MLP for data (404,13) 
model2.add(Dense(units=300, activation='relu', input_dim=X2_train.shape[1]))
# add output layer
model2.add(Dense(units=1, activation='relu'))
# compile regression model loss should be mean_squared_error //
model2.compile(optimizer="adam", loss="mean_squared_error", metrics=["mean_squared_error"])
# fit model
model2.fit(X2_train, y2_train, epochs=200, batch_size=32, validation_split=0.2,verbose=0)

MODEL 3

In [27]:
np.random.seed(7)
# set the TF seed
set_random_seed(7)
# built Keras sequential model 
model3 = Sequential()
# add batch normalization
model3.add(BatchNormalization())
# add layer to the MLP for data (404,13) 
model3.add(Dense(units=300, activation='relu', input_dim=X3_train.shape[1]))
# add output layer
model3.add(Dense(units=1, activation='relu'))
# compile regression model loss should be mean_squared_error //
model3.compile(optimizer="adam", loss="mean_squared_error", metrics=["mean_squared_error"])
# fit model
model3.fit(X3_train, y3_train, epochs=200, batch_size=32, validation_split=0.2,verbose=0)


MODEL0 EVALUATION

In [28]:
pred0 =model0.predict(X0_test)
print('Model0 Evaluation on test set:')
print("Mean squared error (MSE):       %f" % sklearn.metrics.mean_squared_error(y0_test,pred0))
print("R square (R^2):                 %f" % sklearn.metrics.r2_score(y0_test,pred0))
print("\n")
print('Model0 Evaluation on Validation set:')
valid_pred0= model0.predict(X_test_val)
print("Mean squared error (MSE):       %f" % sklearn.metrics.mean_squared_error(y_test_val,valid_pred0))
print("R square (R^2):                 %f" % sklearn.metrics.r2_score(y_test_val,valid_pred0))

Model0 Evaluation on test set:
Mean squared error (MSE):       1.266393
R square (R^2):                 0.260944


Model0 Evaluation on Validation set:
Mean squared error (MSE):       1.008466
R square (R^2):                 0.199456


MODEL1 EVALUATION

In [29]:
pred1 =model1.predict(X1_test) 
print('Model1 Evaluation on test set:')
print("Mean squared error (MSE):       %f" % sklearn.metrics.mean_squared_error(y1_test,pred1))
print("R square (R^2):                 %f" % sklearn.metrics.r2_score(y1_test,pred1))
print("\n")
print('Model1 Evaluation on Validation set:')
valid_pred1 = model1.predict(X_test_val)
print("Mean squared error (MSE):       %f" % sklearn.metrics.mean_squared_error(y_test_val,valid_pred1))
print("R square (R^2):                 %f" % sklearn.metrics.r2_score(y_test_val,valid_pred1))

Model1 Evaluation on test set:
Mean squared error (MSE):       0.517066
R square (R^2):                 0.616013


Model1 Evaluation on Validation set:
Mean squared error (MSE):       0.820685
R square (R^2):                 0.348521


MODEL 2 EVALUATION

In [30]:
# Predict test set
print('Model2 Test:')
y2_pred = model2.predict(X2_test) 
print("Mean squared error (MSE):       %f" % sklearn.metrics.mean_squared_error(y2_test,y2_pred))
print("R square (R^2):                 %f" % sklearn.metrics.r2_score(y2_test,y2_pred))
print("\n")
#Evaluate model2 on validation prediction
print('Model2 Validation:')
valid_pred2 = model2.predict(X_test_val)
print("Mean squared error (MSE):       %f" % sklearn.metrics.mean_squared_error(y_test_val,valid_pred2))
print("R square (R^2):                 %f" % sklearn.metrics.r2_score(y_test_val,valid_pred2))



Model2 Test:
Mean squared error (MSE):       0.385953
R square (R^2):                 0.737140


Model2 Validation:
Mean squared error (MSE):       0.683303
R square (R^2):                 0.457577


MODEL 3 EVALUATION

In [31]:
# Predict test set
y3_pred = model3.predict(X3_test) 
print('Test Set:')
print("Mean squared error (MSE):       %f" % sklearn.metrics.mean_squared_error(y3_test,y3_pred))
print("R square (R^2):                 %f" % sklearn.metrics.r2_score(y3_test,y3_pred))
print("\n")
#Evaluate model3 on validation prediction
print('Validation set:')
valid_pred3 = model3.predict(X_test_val)
print("Mean squared error (MSE):       %f" % sklearn.metrics.mean_squared_error(y_test_val,valid_pred3))
print("R square (R^2):                 %f" % sklearn.metrics.r2_score(y_test_val,valid_pred3))

Test Set:
Mean squared error (MSE):       0.370149
R square (R^2):                 0.772407


Validation set:
Mean squared error (MSE):       0.627490
R square (R^2):                 0.501883
